In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.2 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=f1a82417a449a6ec788718374ca1e8a547fbba136fb3e5876c41d58d91e44dd5
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.3 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 ccc_compiled.csv	       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_601 = 'csv/US06_02.csv'

In [8]:
ct = pd.read_csv(ct_path_601)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,101599653296812_2902484076541675,Facebook,2020-06-02 22:25:16,2021-07-22 18:49:53,link,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,"WHY THE DOUBLE STANDARD, CAN CHINA NOW RESTORE...",1.0,https://www.theguardian.com/us-news/2020/jun/0...,https://www.facebook.com/101599653296812/posts...,1713,-0.888889,1.0,0,1,0,0,0,0,0,0,0,0,2,1,2,1,1,1,0,1,0,0,7973554,The Republican Revolt,the.republican.revolt,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Throughout the centuries there were men and wo...,False,2012-01-15 04:44:53,en,0,7973554|2902484076541675
1,651059541700415_1766972923442399,Facebook,2020-06-02 22:25:13,2021-07-20 09:48:14,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,“Gonzaga stands in solidarity with all those w...,2.0,https://www.facebook.com/ZagAlumni/photos/a.65...,https://www.facebook.com/651059541700415/posts...,2088,1.684211,1.0,26,3,1,2,0,0,0,0,0,1,5,2,2,2,0,0,0,0,0,1,1950055,Gonzaga University Alumni Association,ZagAlumni,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Welcome to the official Zag Alumni & Friends p...,False,2016-01-06 19:12:27,en,0,1950055|1766972923442399
2,133740559987144_3453728487988318,Facebook,2020-06-02 22:25:13,2021-06-23 21:32:13,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,The Native American Rights Fund stands in soli...,1.0,https://www.facebook.com/NativeAmericanRightsF...,https://www.facebook.com/133740559987144/posts...,37752,1.496124,1.0,107,36,3,46,0,0,1,0,0,11,76,20,5,16,2,2,5,3,0,5,1842925,Native American Rights Fund,NativeAmericanRightsFund,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 1970, the Native American Rights Fu...",True,2010-06-28 16:20:54,en,0,1842925|3453728487988318
3,188000221228143_3507581695936629,Facebook,2020-06-02 22:25:11,2021-08-29 23:24:06,link,BLACKOUT TUESDAY: You CAN stand against racism...,kfyi.iheart.com,Glenn makes it unequivocally clear: he's GLAD ...,Glenn makes it unequivocally clear: he's GLAD ...,1.0,http://ihe.art/sa6SA6f,https://www.facebook.com/188000221228143/posts...,15829,-2.071429,1.0,12,1,1,0,0,0,0,0,0,0,3,2,4,3,2,5,5,5,0,2,38460,550 KFYI,550kfyi,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,NewsTalk 550 KFYI - The Valley's Talk Station....,True,2011-01-21 20:19:39,en,0,38460|3507581695936629
4,10716057521_10157325690307522,Facebook,2020-06-02 22:25:09,2021-08-13 11:34:05,link,Watch The New York Times' Nikole Hannah-Jones ...,theweek.com,It's been a week since George Floyd's death in...,"""We want to always say that slavery was a long...",1.0,https://theweek.com/speedreads/917864/watch-ne...,https://www.facebook.com/10716057521/posts/101...,419148,1.191176,1.0,39,19,6,1,2,0,14,0,0,0,20,5,13,6,3,9,5,7,0,5,15634,THE WEEK,theweek,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,All you need to know about everything that mat...,True,2008-02-26 22:32:05,en,0,15634|10157325690307522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'HONG KONG': 1, 'Russia': 2, 'US': 4, 'China'...","{'US': 1, 'Hong Kong’s': 1}"
1,{},"{'US': 1, 'Hong Kong’s': 1}"
2,{},"{'US': 1, 'Hong Kong’s': 1}"
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0602.csv')

In [25]:
ct=pd.read_csv('a0602.csv')

In [26]:
!ls

 a0529-a0611_UK		       dataframe_result_csv
 a0529-a0611_US		       dataframe_result_csv_EU
 a0529.csv		       dataframe_result_csv_UK
 a0530.csv		       Github
 a0601.csv		       heatmap529_UK.ipynb
 a0602.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'


In [27]:
###
#ct=pd.read_csv('a0601.csv')

In [28]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,101599653296812_2902484076541675,Facebook,2020-06-02 22:25:16,2021-07-22 18:49:53,link,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,"WHY THE DOUBLE STANDARD, CAN CHINA NOW RESTORE...",1.0,https://www.theguardian.com/us-news/2020/jun/0...,https://www.facebook.com/101599653296812/posts...,1713,-0.888889,1.0,0,1,0,0,0,0,0,0,0,0,2,1,2,1,1,1,0,1,0,0,7973554,The Republican Revolt,the.republican.revolt,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Throughout the centuries there were men and wo...,False,2012-01-15 04:44:53,en,0,7973554|2902484076541675,"{'HONG KONG': 1, 'Russia': 2, 'US': 4, 'China'...","{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
1,1,651059541700415_1766972923442399,Facebook,2020-06-02 22:25:13,2021-07-20 09:48:14,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,“Gonzaga stands in solidarity with all those w...,2.0,https://www.facebook.com/ZagAlumni/photos/a.65...,https://www.facebook.com/651059541700415/posts...,2088,1.684211,1.0,26,3,1,2,0,0,0,0,0,1,5,2,2,2,0,0,0,0,0,1,1950055,Gonzaga University Alumni Association,ZagAlumni,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Welcome to the official Zag Alumni & Friends p...,False,2016-01-06 19:12:27,en,0,1950055|1766972923442399,{},"{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
2,2,133740559987144_3453728487988318,Facebook,2020-06-02 22:25:13,2021-06-23 21:32:13,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,The Native American Rights Fund stands in soli...,1.0,https://www.facebook.com/NativeAmericanRightsF...,https://www.facebook.com/133740559987144/posts...,37752,1.496124,1.0,107,36,3,46,0,0,1,0,0,11,76,20,5,16,2,2,5,3,0,5,1842925,Native American Rights Fund,NativeAmericanRightsFund,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 1970, the Native American Rights Fu...",True,2010-06-28 16:20:54,en,0,1842925|3453728487988318,{},"{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
3,3,188000221228143_3507581695936629,Facebook,2020-06-02 22:25:11,2021-08-29 23:24:06,link,BLACKOUT TUESDAY: You CAN stand against racism...,kfyi.iheart.com,Glenn makes it unequivocally clear: he's GLAD ...,Glenn makes it unequivocally clear: he's GLAD ...,1.0,http://ihe.art/sa6SA6f,https://www.facebook.com/188000221228143/posts...,15829,-2.071429,1.0,12,1,1,0,0,0,0,0,0,0,3,2,4,3,2,5,5,5,0,2,38460,550 KFYI,550kfyi,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,NewsTalk 550 KFYI - The Valley's Talk Station....,True,2011-01-21 20:19:39,en,0,38460|3507581695936629,{},{},{}
4,4,10716057521_10157325690307522,Facebook,2020-06-02 22:25:09,2021-08-13 11:34:05,link,Watch The New York Times' Nikole Hannah-Jones ...,theweek.com,It's been a week since George Floyd's death in...,"""We want to always say that slavery was a long...",1.0,https://theweek.com/speedreads/917864/watch-ne...,https://www.facebook.com/10716057521/posts/101...,419148,1.191176,1.0,39,19,6,1,2,0,14,0,0,0,20,5,13,6,3,9,5,7,0,5,15634,THE WEEK,theweek,https://scontent-sea1-1.xx.

In [29]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [30]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [31]:
###
ct_ner = ct

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'HONG KONG': 1, 'Russia': 2, 'US': 4, 'China'...","{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
1,{},"{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
2,{},"{'US': 1, 'Hong Kong’s': 1}","{'China': 1, 'US': 1}"
3,{},{},{}
4,{},{},{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [34]:
ct_ner['NER-msg']

0        [Hong Kong, Constitutional Republic, America, ...
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
28378                                                   []
28379                                           [St. Paul]
28380                                         [Fort Worth]
28381                             [West Palm Beach Police]
28382                                            [Atlanta]
Name: NER-msg, Length: 28383, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [36]:
ct_ner['NER']

0        [Hong Kong, Constitutional Republic, America, ...
1                             [Hong Kong’s, US, China, US]
2                             [Hong Kong’s, US, China, US]
3                                                       []
4                                                       []
                               ...                        
28378                                    [Omaha, Nebraska]
28379                       [St. Paul, St. Paul, St. Paul]
28380                 [Fort Worth, Fort Worth, Fort Worth]
28381    [West Palm Beach Police, Minneapolis, WEST PAL...
28382    [Atlanta, Atlanta, ATLANTA, America, Minneapol...
Name: NER, Length: 28383, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [39]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,101599653296812_2902484076541675,Facebook,2020-06-02 22:25:16,2021-07-22 18:49:53,link,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,"WHY THE DOUBLE STANDARD, CAN CHINA NOW RESTORE...",1.0,https://www.theguardian.com/us-news/2020/jun/0...,https://www.facebook.com/101599653296812/posts...,1713,-0.888889,1.0,0,1,0,0,0,0,0,0,0,0,2,1,2,1,1,1,0,1,0,0,7973554,The Republican Revolt,the.republican.revolt,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Throughout the centuries there were men and wo...,False,2012-01-15 04:44:53,en,0,7973554|2902484076541675,"[Hong Kong, Constitutional Republic, America, ...","[Hong Kong’s, US]","[China, US]","[hong kong’s, hong kong, constitutional republ..."
1,1,651059541700415_1766972923442399,Facebook,2020-06-02 22:25:13,2021-07-20 09:48:14,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,“Gonzaga stands in solidarity with all those w...,2.0,https://www.facebook.com/ZagAlumni/photos/a.65...,https://www.facebook.com/651059541700415/posts...,2088,1.684211,1.0,26,3,1,2,0,0,0,0,0,1,5,2,2,2,0,0,0,0,0,1,1950055,Gonzaga University Alumni Association,ZagAlumni,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Welcome to the official Zag Alumni & Friends p...,False,2016-01-06 19:12:27,en,0,1950055|1766972923442399,[],"[Hong Kong’s, US]","[China, US]","[china, hong kong’s, us]"
2,2,133740559987144_3453728487988318,Facebook,2020-06-02 22:25:13,2021-06-23 21:32:13,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,The Native American Rights Fund stands in soli...,1.0,https://www.facebook.com/NativeAmericanRightsF...,https://www.facebook.com/133740559987144/posts...,37752,1.496124,1.0,107,36,3,46,0,0,1,0,0,11,76,20,5,16,2,2,5,3,0,5,1842925,Native American Rights Fund,NativeAmericanRightsFund,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 1970, the Native American Rights Fu...",True,2010-06-28 16:20:54,en,0,1842925|3453728487988318,[],"[Hong Kong’s, US]","[China, US]","[china, hong kong’s, us]"
3,3,188000221228143_3507581695936629,Facebook,2020-06-02 22:25:11,2021-08-29 23:24:06,link,BLACKOUT TUESDAY: You CAN stand against racism...,kfyi.iheart.com,Glenn makes it unequivocally clear: he's GLAD ...,Glenn makes it unequivocally clear: he's GLAD ...,1.0,http://ihe.art/sa6SA6f,https://www.facebook.com/188000221228143/posts...,15829,-2.071429,1.0,12,1,1,0,0,0,0,0,0,0,3,2,4,3,2,5,5,5,0,2,38460,550 KFYI,550kfyi,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,NewsTalk 550 KFYI - The Valley's Talk Station....,True,2011-01-21 20:19:39,en,0,38460|3507581695936629,[],[],[],[]
4,4,10716057521_10157325690307522,Facebook,2020-06-02 22:25:09,2021-08-13 11:34:05,link,Watch The New York Times' Nikole Hannah-Jones ...,theweek.com,It's been a week since George Floyd's death in...,"""We want to always say that slavery was a long...",1.0,https://theweek.com/speedreads/917864/watch-ne...,https://www.facebook.com/10716057521/posts/101...,419148,1.191176,1.0,39,19,6,1,2,0,14,0,0,0,20,5,13,6,3,9,5,7,0

In [40]:
ct_ner['NER-1']=ct_ner['NER']

In [41]:
ct_ner['NER-1'][0]

['hong kong’s',
 'hong kong',
 'constitutional republic',
 'america',
 'hong kong',
 'russia',
 'china',
 'us']

In [42]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [43]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,101599653296812_2902484076541675,Facebook,2020-06-02 22:25:16,2021-07-22 18:49:53,link,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,"WHY THE DOUBLE STANDARD, CAN CHINA NOW RESTORE...",1.0,https://www.theguardian.com/us-news/2020/jun/0...,https://www.facebook.com/101599653296812/posts...,1713,-0.888889,1.0,0,1,0,0,0,0,0,0,0,0,2,1,2,1,1,1,0,1,0,0,7973554,The Republican Revolt,the.republican.revolt,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Throughout the centuries there were men and wo...,False,2012-01-15 04:44:53,en,0,7973554|2902484076541675,"[Hong Kong, Constitutional Republic, America, ...","[Hong Kong’s, US]","[China, US]","[hong kong’s, hong kong, constitutional republ...","[hong kong’s, hong kong, constitutional republ..."
1,1,651059541700415_1766972923442399,Facebook,2020-06-02 22:25:13,2021-07-20 09:48:14,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,“Gonzaga stands in solidarity with all those w...,2.0,https://www.facebook.com/ZagAlumni/photos/a.65...,https://www.facebook.com/651059541700415/posts...,2088,1.684211,1.0,26,3,1,2,0,0,0,0,0,1,5,2,2,2,0,0,0,0,0,1,1950055,Gonzaga University Alumni Association,ZagAlumni,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Welcome to the official Zag Alumni & Friends p...,False,2016-01-06 19:12:27,en,0,1950055|1766972923442399,[],"[Hong Kong’s, US]","[China, US]","[china, hong kong’s, us]","[china, hong kong’s, us]"
2,2,133740559987144_3453728487988318,Facebook,2020-06-02 22:25:13,2021-06-23 21:32:13,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,The Native American Rights Fund stands in soli...,1.0,https://www.facebook.com/NativeAmericanRightsF...,https://www.facebook.com/133740559987144/posts...,37752,1.496124,1.0,107,36,3,46,0,0,1,0,0,11,76,20,5,16,2,2,5,3,0,5,1842925,Native American Rights Fund,NativeAmericanRightsFund,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 1970, the Native American Rights Fu...",True,2010-06-28 16:20:54,en,0,1842925|3453728487988318,[],"[Hong Kong’s, US]","[China, US]","[china, hong kong’s, us]","[china, hong kong’s, us]"
3,3,188000221228143_3507581695936629,Facebook,2020-06-02 22:25:11,2021-08-29 23:24:06,link,BLACKOUT TUESDAY: You CAN stand against racism...,kfyi.iheart.com,Glenn makes it unequivocally clear: he's GLAD ...,Glenn makes it unequivocally clear: he's GLAD ...,1.0,http://ihe.art/sa6SA6f,https://www.facebook.com/188000221228143/posts...,15829,-2.071429,1.0,12,1,1,0,0,0,0,0,0,0,3,2,4,3,2,5,5,5,0,2,38460,550 KFYI,550kfyi,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,NewsTalk 550 KFYI - The Valley's Talk Station....,True,2011-01-21 20:19:39,en,0,38460|3507581695936629,[],[],[],[],[]
4,4,10716057521_10157325690307522,Facebook,2020-06-02 22:25:09,2021-08-13 11:34:05,link,Watch The New York Times' Nikole Hannah-Jones ...,theweek.com,It's been a week since George Floyd's death in...,"""We want to always say that slavery was a long...",1.0,https://theweek.com/speedreads/917864/watch-ne.

In [44]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [45]:
!ls

 Country.txt			 dataframe0610_post.csv
 dataframe0529_post.csv		 dataframe0610_post_remove.csv
 dataframe0529_post_remove.csv	 dataframe0611_post.csv
 dataframe0530_post.csv		 dataframe0611_post_remove.csv
 dataframe0530_post_remove.csv	 dataframe_mapping
 dataframe0531_post.csv		 dataframe_mapping_0529.csv
 dataframe0531_post_remove.csv	 dataframe_mapping_0530.csv
 dataframe0601_post.csv		 dataframe_mapping_0531.csv
 dataframe0601_post_remove.csv	 dataframe_mapping_0601.csv
 dataframe0602_post.csv		 dataframe_mapping_0602.csv
 dataframe0602_post_remove.csv	 dataframe_mapping_0603.csv
 dataframe0603_post.csv		 dataframe_mapping_0604.csv
 dataframe0603_post_remove.csv	 dataframe_mapping_0605.csv
 dataframe0604_post.csv		 dataframe_mapping_0606.csv
 dataframe0604_post_remove.csv	 dataframe_mapping_0607.csv
 dataframe0605_post.csv		 dataframe_mapping_0608.csv
 dataframe0605_post_remove.csv	 dataframe_mapping_0609.csv
 dataframe0606_post.csv		 dataframe_mapping_0610.csv
 dataframe0606_post

In [46]:
ct_mapping=pd.read_csv('dataframe_mapping_0602.csv',encoding='ISO-8859-1')

In [47]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,6392,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,2182,"Minnesota, United States",45.989659,-94.611329
2,2,america,2144,"America, Horst aan de Maas, Limburg, Nederland",51.437148,5.979900
3,3,the united states,1191,"The Summit, Guadalupe County, Texas, United St...",29.633008,-97.988594
4,4,u.s.,1147,"Unterer See, Kirchheim unter Teck, Vereinbarte...",48.630579,9.417545
...,...,...,...,...,...,...
3161,3161,the city and county building,1,"Northern and Shell, 10, Lower Thames Street, C...",51.509031,-0.084806
3162,3162,gabriela dc,1,"Rua DC Dez, Campo Limpo, Pampalona, Feira de S...",-12.224429,-38.974255
3163,3163,anakbayan dc,1,NaN,NaN,NaN
3164,3164,migrante dc,1,"Fundacion De Atencion Al Migrante, Avenida Cal...",4.640051,-74.116451


In [48]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [49]:
country_list['Name']

0            afghanistan
1          �land islands
2                albania
3                algeria
4         american samoa
             ...        
250                   us
251    the united states
252              america
253                  u.s
254        united states
Name: Name, Length: 255, dtype: object

In [50]:
states_list=pd.read_csv('states.csv')
states_list['State']

0                             Alabama, United States
1                              Alaska, United States
2                             Arizona, United States
3                            Arkansas, United States
4                          California, United States
                           ...                      
165         Armagh, Northern Ireland, United Kingdom
166           Down, Northern Ireland, United Kingdom
167      Fermanagh, Northern Ireland, United Kingdom
168    Londonderry, Northern Ireland, United Kingdom
169         Tyrone, Northern Ireland, United Kingdom
Name: State, Length: 170, dtype: object

In [51]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [52]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [53]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [54]:
pd.set_option('display.max_rows', None)

In [55]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,6392,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,2182,NaN,45.989659,-94.611329
2,2,america,2144,NaN,51.437148,5.979900
3,3,the united states,1191,NaN,29.633008,-97.988594
4,4,u.s.,1147,NaN,48.630579,9.417545
5,5,us,988,NaN,39.783730,-100.445882
6,6,houston,967,"Houston, Harris County, Texas, 77002-5902, Uni...",29.758938,-95.367697
7,7,atlanta,837,"Atlanta, Fulton County, Georgia, 30303-3506, U...",33.748992,-84.390264
8,8,washington,747,"Washington, District of Columbia, United States",38.895037,-77.036543
9,9,chicago,640,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421


In [56]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,6392,"Minneapolis, Hennepin County, Minnesota, 55488...",44.977300,-93.265469
1,1,minnesota,2182,NaN,45.989659,-94.611329
2,2,america,2144,NaN,51.437148,5.979900
3,3,the united states,1191,NaN,29.633008,-97.988594
4,4,u.s.,1147,NaN,48.630579,9.417545
5,5,us,988,NaN,39.783730,-100.445882
6,6,houston,967,"Houston, Harris County, Texas, 77002-5902, Uni...",29.758938,-95.367697
7,7,atlanta,837,"Atlanta, Fulton County, Georgia, 30303-3506, U...",33.748992,-84.390264
8,8,washington,747,"Washington, District of Columbia, United States",38.895037,-77.036543
9,9,chicago,640,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421


In [57]:
ct_mapping.to_csv('dataframe_mapping_0602.csv')

In [58]:
#ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [59]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [60]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,101599653296812_2902484076541675,Facebook,2020-06-02 22:25:16,2021-07-22 18:49:53,link,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,"WHY THE DOUBLE STANDARD, CAN CHINA NOW RESTORE...",1.0,https://www.theguardian.com/us-news/2020/jun/0...,https://www.facebook.com/101599653296812/posts...,1713,-0.888889,1.0,0,1,0,0,0,0,0,0,0,0,2,1,2,1,1,1,0,1,0,0,7973554,The Republican Revolt,the.republican.revolt,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Throughout the centuries there were men and wo...,False,2012-01-15 04:44:53,en,0,7973554|2902484076541675,"[Hong Kong, Constitutional Republic, America, ...","[Hong Kong’s, US]","[China, US]","[hong kong’s, hong kong, constitutional republ...","[nan, nan, nan, Ð Ð¾ÑÑÐ¸Ñ, nan, nan]"
1,1,651059541700415_1766972923442399,Facebook,2020-06-02 22:25:13,2021-07-20 09:48:14,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,“Gonzaga stands in solidarity with all those w...,2.0,https://www.facebook.com/ZagAlumni/photos/a.65...,https://www.facebook.com/651059541700415/posts...,2088,1.684211,1.0,26,3,1,2,0,0,0,0,0,1,5,2,2,2,0,0,0,0,0,1,1950055,Gonzaga University Alumni Association,ZagAlumni,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Welcome to the official Zag Alumni & Friends p...,False,2016-01-06 19:12:27,en,0,1950055|1766972923442399,[],"[Hong Kong’s, US]","[China, US]","[china, hong kong’s, us]","[nan, nan]"
2,2,133740559987144_3453728487988318,Facebook,2020-06-02 22:25:13,2021-06-23 21:32:13,photo,"'Mr President, don't go hide': China goads US ...",theguardian.com,Officials and state media appear to revel in s...,The Native American Rights Fund stands in soli...,1.0,https://www.facebook.com/NativeAmericanRightsF...,https://www.facebook.com/133740559987144/posts...,37752,1.496124,1.0,107,36,3,46,0,0,1,0,0,11,76,20,5,16,2,2,5,3,0,5,1842925,Native American Rights Fund,NativeAmericanRightsFund,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 1970, the Native American Rights Fu...",True,2010-06-28 16:20:54,en,0,1842925|3453728487988318,[],"[Hong Kong’s, US]","[China, US]","[china, hong kong’s, us]","[nan, nan]"
3,3,188000221228143_3507581695936629,Facebook,2020-06-02 22:25:11,2021-08-29 23:24:06,link,BLACKOUT TUESDAY: You CAN stand against racism...,kfyi.iheart.com,Glenn makes it unequivocally clear: he's GLAD ...,Glenn makes it unequivocally clear: he's GLAD ...,1.0,http://ihe.art/sa6SA6f,https://www.facebook.com/188000221228143/posts...,15829,-2.071429,1.0,12,1,1,0,0,0,0,0,0,0,3,2,4,3,2,5,5,5,0,2,38460,550 KFYI,550kfyi,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,NewsTalk 550 KFYI - The Valley's Talk Station....,True,2011-01-21 20:19:39,en,0,38460|3507581695936629,[],[],[],[],[]
4,4,10716057521_10157325690307522,Facebook,2020-06-02 22:25:09,2021-08-13 11:34:05,link,Watch The New York Times' Nikole Hannah-Jones ...,theweek.com,It's been a week since George Floyd's death in...,"""We want to always say that slavery was a long...",1.0,https://theweek.com/speedreads/917864/watch-ne...,https://www.facebook.com/1071605752

In [61]:
ct_ner['NER-1'][:100]

0               [nan, nan, nan, Ð Ð¾ÑÑÐ¸Ñ, nan, nan]
1                                            [nan, nan]
2                                            [nan, nan]
3                                                    []
4                                                    []
5                                                 [nan]
6                                                    []
7                                                    []
8     [Prescott, Yavapai County, Arizona, United Sta...
9     [Prescott, Yavapai County, Arizona, United Sta...
10                                                [nan]
11    [Selma, Dallas County, Alabama, 36701, United ...
12    [Minneapolis, Hennepin County, Minnesota, 5548...
13    [Las Vegas, Clark County, Nevada, United State...
14              [Harrisonburg, Virginia, United States]
15              [Harrisonburg, Virginia, United States]
16    [Salem, Marion County, Oregon, United States, ...
17      [Chicago, Cook County, Illinois, United 

In [62]:
ct_ner.to_csv('dataframe0602_post_remove.csv')

In [63]:
!ls

 Country.txt			 dataframe0610_post.csv
 dataframe0529_post.csv		 dataframe0610_post_remove.csv
 dataframe0529_post_remove.csv	 dataframe0611_post.csv
 dataframe0530_post.csv		 dataframe0611_post_remove.csv
 dataframe0530_post_remove.csv	 dataframe_mapping
 dataframe0531_post.csv		 dataframe_mapping_0529.csv
 dataframe0531_post_remove.csv	 dataframe_mapping_0530.csv
 dataframe0601_post.csv		 dataframe_mapping_0531.csv
 dataframe0601_post_remove.csv	 dataframe_mapping_0601.csv
 dataframe0602_post.csv		 dataframe_mapping_0602.csv
 dataframe0602_post_remove.csv	 dataframe_mapping_0603.csv
 dataframe0603_post.csv		 dataframe_mapping_0604.csv
 dataframe0603_post_remove.csv	 dataframe_mapping_0605.csv
 dataframe0604_post.csv		 dataframe_mapping_0606.csv
 dataframe0604_post_remove.csv	 dataframe_mapping_0607.csv
 dataframe0605_post.csv		 dataframe_mapping_0608.csv
 dataframe0605_post_remove.csv	 dataframe_mapping_0609.csv
 dataframe0606_post.csv		 dataframe_mapping_0610.csv
 dataframe0606_post